glob은 해당하는 파일을 찾아줍니다. ../은 상위 폴더 주소 입니다. ./은 현재 폴더 주소입니다. 

실험 파일 전체에 적용되는 값들은 한 쪽 (주로 Jupyter notebook 맨 위쪽)에 몰아두는 것이 실수를 방지할 수 있습니다. (config.py를 만드는 것도 좋은 방법입니다.)

In [1]:
import glob

RAW_DATA_FOLDER = './data/movie_reviews/'
PREPROSSED_DATA_FOLDER = './tmp/'

json_fnames = glob.glob('%s/*.json' % RAW_DATA_FOLDER)
print(len(json_fnames))

217


In [2]:
import json

with open(json_fnames[2], encoding='utf-8') as f:
    json_article = json.load(f)

해당 문서는 네이버 영화의 리뷰를 수집한 것입니다. 이 중에서 우리가 분석에 이용할 텍스트 데이터만을 가지고 오겠습니다. 

코드는 항상 완벽할 수 없습니다. 데이터를 수집할 때 가능하다면 원본의 데이터를 늘 저장하시기 바랍니다. content를 만드는 parser가 잘못 만들어졌다면, 이미 저장된 JSON 파일들을 불러와서 다시 파싱하면 되니까요. 

영화 평점은 개봉 전 / 개봉 후 두 가지로 나뉘어져서 수집하였습니다. 각각의 리뷰는 몇 개씩 있는지 확인합니다. 

In [3]:
json_article.keys()

dict_keys(['comments_after', 'comments_before'])

In [4]:
len(json_article['comments_before']), len(json_article['comments_after'])

(13, 51)

각 comments_before, comments_after의 형식은 list of dict입니다. 하나만 꺼내서 구조를 살펴봅시다. 

In [5]:
type(json_article['comments_before'])

list

In [6]:
type(json_article['comments_before'][2])

dict

영화 평에 대한 agree, disagree, 평점, 평, 사용자아이디, 작성일시가 있습니다. 이 중에서 텍스트만을 가지고 와보겠습니다. 

In [7]:
json_article['comments_before'][2]

{'agree': 2,
 'disagree': 0,
 'score': 1,
 'text': '...제목에서 덕필이 왔다...',
 'user': '1441040',
 'written_at': '2013.10.22 10:28'}

In [8]:
json_article['comments_before'][2]['text']

'...제목에서 덕필이 왔다...'

하나의 JSON에 들어있는 여러 개의 리뷰로부터 텍스트만을 가지고 오는 함수를 만들어 봅시다. for loop을 돌면서 text만을 꺼내서 리스트로 저장하면 됩니다. 

텍스트가 없거나 빈 텍스트일 경우 다음 for loop으로 넘어가기 위하여 if text: 를 넣었습니다. 

In [9]:
texts = []
for comment in json_article['comments_before']:
    text = comment.get('text', '')
    if not text:
        continue
    texts.append(text)
    
print(len(texts))

13


개인적인 선호이지만, a line a document로 텍스트를 저장하는 편입니다. 나중에 term frequency matrix를 만들거나 Word2Vec, Doc2Vec을 학습할 때 편리합니다. 

하지만 하나의 문서는 여러 줄로 구성될 수 있습니다. 그래서 실제 줄바꿈이 있는 경우에는 **두 칸 띄어쓰기**를 이용하여 구분합니다. 기호에 따라서 탭 구분을 하셔도 좋습니다. 이 때, 줄 구분기호 (두 칸 띄어쓰기)는 반드시 한 문장 안에서 없어야 되겠죠. 

그리고 필요한 메타 정보들도 line number를 맞춰서 index 파일로 만드는 것을 선호합니다. 이렇게 line number가 맞으면 필요할 때 해당 문서의 메타 정보들을 이용할 수 있으니까요. 

JSON 문서에 만약 text, written_at과 같은 정보가 존재하지 않아 오류가 날 것을 대비하여 dict.get(key, '')를 이용하여 json parser를 만듭니다. Python의 함수는 한 번에 여러 개의 변수가 return 될 수 있습니다.

In [10]:
import json

def parse_text_from_json(json_fname):
    with open(json_fname, encoding='utf-8') as f:        
        json_object = json.load(f)
        texts = []
        for key in ['comments_before', 'comments_after']:
            for comment in json_object.get(key, []):                
                text = comment.get('text', '')
                if not text:
                    continue
                texts.append(text)
        return texts
    
texts = parse_text_from_json(json_fnames[2])
len(texts)

64

위 파일을 tmp/test_reviews.txt로 저장해봅시다. 

In [11]:
with open('%s/test_reviews.txt' % PREPROSSED_DATA_FOLDER, 'w', encoding='utf-8') as f:
    for text in texts:
        f.write('%s\n' % text)

평점과 텍스트를 탭으로 분리하여 하나의 파일에 함께 저장하고 싶다면 parser를 아래처럼 만들 수 있습니다. 

In [12]:
import json

def parse_text_and_score_from_json(json_fname):
    with open(json_fname, encoding='utf-8') as f:        
        json_object = json.load(f)
        texts = []
        scores = []
        for key in ['comments_before', 'comments_after']:
            for comment in json_object.get(key, []):                
                text = comment.get('text', '')
                score = comment.get('score', -1)
                if (not text) or (score == -1):
                    continue
                texts.append(text)
                scores.append(score)
        return texts, scores
    
texts, scores = parse_text_and_score_from_json(json_fnames[2])
len(texts), len(scores)

(64, 64)

zip은 두 개 이상의 list (iterable한 set도 가능)를 함께 for loop을 돌 수 있도록 해줍니다. 만약 list의 길이가 다르다면 짧은 것에서 멈춥니다. 

In [13]:
for text, score in zip(texts[5:9], scores[5:]):
    print(text, score)

무섭다.. 공포영화 1순위 ㄱㄱ싱! 9
왠지 웃기기는 할듯. 일단 봐야 알겠음 9
피판에서 봤는데, 역시 꽃과뱀 감독답게 아주 아름답고 좋았습니다.. 10
애니메이션 설정을 영화로 표현하는건 좀 그러네요. 현실감이없어 일본은.... 1


모든 파일들을 돌면서 하나의 파일에 평점과 텍스트를 저장하겠습니다. 

    '%d\t%s' % (score, text)
    
위 부분은 %d에 int를, %s에 str을 넣겠다는 의미입니다. 

In [14]:
'%d/t%s' % (scores[5], texts[5])

'9/t무섭다.. 공포영화 1순위 ㄱㄱ싱!'

현재 영화 파일의 이름을 알아내기 위해서는 str.split()을 하면 됩니다. 

In [15]:
print(json_fnames[2])
print(json_fnames[2].split('/')[-1])
print(json_fnames[2].split('/')[-1].split('.')[0])

./data/movie_reviews/100084.json
100084.json
100084


In [16]:
with open('%s/reviews.txt' % PREPROSSED_DATA_FOLDER, 'w', encoding='utf-8') as f:
    for json_fname in json_fnames:
        movie_idx = json_fname.split('/')[-1].split('.')[0]
        texts, scores = parse_text_and_score_from_json(json_fname)
        for score, text in zip(scores, texts):
            f.write('%s\t%d\t%s\n' % (movie_idx, score, text.replace('\t', ' ').replace('\n',' ')))